<a href="https://colab.research.google.com/github/pramodks79/KnowledgeGraph_DataCatalog/blob/main/JMSAI_DataCatalog_KnowledgeGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Build KnowledgeGraph(KG) for the DataCatalog**


In [ ]:
!pip install --quiet langchain-community langchain-experimental langchain_google_genai json-repair

# Building knowledge graphs with LLMGraph Transformer, using Gemini Models


In [ ]:
# Getting  the Model Names and Key from the Secretstore
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata # this is to read key from the Google Colab environment.

# 1. Setup Gemini model
os.environ["GOOGLE_API_KEY"] = userdata.get('pkst_apikey')
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0)

## This set up is required if you like to store the data into the datbase - neo4j
The easiest way to get started is to use a free instance of Neo4j Aura, which offers cloud instances of the Neo4j database.

In [ ]:
# Install this package if you like to use neo4j to store the graph, otherwise skip it.
!pip install --quiet neo4j

In [ ]:
# This functionality is required if you like to save the Knowledge Graph in database
from langchain_community.graphs import Neo4jGraph

neo4jurl = userdata.get('neo4jurl')  # From Google Colab
neo4jgraph = Neo4jGraph(
    url=userdata.get('neo4jurl'),
    username=userdata.get('neo4jusername'),
    password=userdata.get('neo4jpassword'),
    refresh_schema=True
)

def clean_graph():
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    neo4jgraph.query(query)

## Set up to visualize the Graph - This function to visualize the graph by generating HTML File


In [ ]:
# Install this library if you like to visualize this graph in noteboook
!pip install --quiet pyvis

In [ ]:
from pyvis.network import Network

def visualize_knowledge_graph(graph_documents):

    # Creating network
    net = Network(height="1200px", width="100%", directed=True,
                      notebook=False, bgcolor="#222222", font_color="white")

    nodes = graph_documents[0].nodes
    relationships = graph_documents[0].relationships

    # Build lookup for valid nodes
    node_dict = {node.id: node for node in nodes}

    # Filter out invalid edges and collect valid node IDs
    valid_edges = []
    valid_node_ids = set()
    for rel in relationships:
        if rel.source.id in node_dict and rel.target.id in node_dict:
            valid_edges.append(rel)
            valid_node_ids.update([rel.source.id, rel.target.id])


    # Track which nodes are part of any relationship
    connected_node_ids = set()
    for rel in relationships:
        connected_node_ids.add(rel.source.id)
        connected_node_ids.add(rel.target.id)

    # Add valid nodes
    for node_id in valid_node_ids:
        node = node_dict[node_id]
        try:
            net.add_node(node.id, label=node.id, title=node.type, group=node.type)
        except:
            continue  # skip if error

    # Add valid edges
    for rel in valid_edges:
        try:
            net.add_edge(rel.source.id, rel.target.id, label=rel.type.lower())
        except:
            continue  # skip if error

    # Configure physics
    net.set_options("""
            {
                "physics": {
                    "forceAtlas2Based": {
                        "gravitationalConstant": -100,
                        "centralGravity": 0.01,
                        "springLength": 200,
                        "springConstant": 0.08
                    },
                    "minVelocity": 0.75,
                    "solver": "forceAtlas2Based"
                }
            }
            """)

    output_file = "datacatalog_knowledge_graph.html"
    net.save_graph(output_file)
    print(f"Graph saved to {os.path.abspath(output_file)}")

    # Try to open in browser
    try:
        import webbrowser
        webbrowser.open(f"file://{os.path.abspath(output_file)}")
    except:
        print("Could not open browser automatically")



## === End code for visualize Graph ==

## **Option 1: Create Graph with LLMGraphTransformer with defined schema. Just visualizing the output, not storing into the database**





In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_core.documents import Document
 # 2. Define a strict schema for the KG to avoid messy extraction
llm_transformer = LLMGraphTransformer(
    llm=llm,
    allowed_nodes=["Table", "Column", "Database", "User"],
    allowed_relationships=["CONTAINS", "REFERENCES", "OWNED_BY", "JOINS_WITH"],
    strict_mode=True
)

## Use the LLMGraphTransformer to generate KG and visualize

In [ ]:
def generate_knowledgeGraph_and_visualize(schema_metadata):
    documents = [Document(page_content=schema_metadata)]
    graph_documents = llm_transformer.convert_to_graph_documents(documents)
    # This function will create HTML file for the graph and save it
    visualize_knowledge_graph(graph_documents)


In [ ]:
generate_knowledgeGraph_and_visualize(schema_metadata_0)

Graph saved to /content/datacatalog_knowledge_graph.html


## **Option 2: Create Graph with LLMGraphTransformer. And storing it into neo4j database**

In [ ]:
llm_transformer_neo4j = LLMGraphTransformer(
    llm=llm
)

In [ ]:
def generate_knowledgeGraph_and_save_in_neo4jdb(schema_metadata):
    documents = [Document(page_content=schema_metadata)]
    graph_documents = llm_transformer_neo4j.convert_to_graph_documents(documents)

    neo4jgraph.add_graph_documents(graph_documents)

In [ ]:
generate_knowledgeGraph_and_save_in_neo4jdb(schema_metadata_2)
print("Graph saved to Neo4j")

Graph saved to Neo4j


In [ ]:
# clean the old graph
clean_graph()

### Following are some sample schema matadaat to test

In [ ]:
schema_metadata_0 = """
Database: PRODUCTION_DB
Table: USERS
Description: Stores customer profile information.
Columns:
 - user_id (INT, PK): Unique identifier
 - email (STRING): Customer email address
 - signup_date (DATE)
Relationships:
 - user_id references ORDERS.customer_id
"""

In [ ]:
# 3. Simulate your Data Catalog Metadata (usually fetched via API)
schema_metadata_1 = """
Database: ECOMMERCE_PROD

Table: USERS
Description: Contains sensitive customer profile data. Owned by Marketing.
Columns:
 - user_id (PK)
 - email (PII)
 - location_code

Table: ORDERS
Description: Records all customer transactions. Owned by Sales.
Columns:
 - order_id (PK)
 - customer_id (FK): References USERS.user_id
 - product_id (FK): References PRODUCTS.sku
 - order_total (DECIMAL)

Table: PRODUCTS
Description: Master catalog of available items.
Columns:
 - sku (PK): Unique product identifier
 - product_name (STRING)
 - category (STRING)
"""

In [ ]:
schema_metadata_2 = """
### ENTITIES AND ATTRIBUTES
- NODE Table: USERS {description: "Master customer profile table", owner: "Marketing"}
  - COLUMN: user_id (Type: INTEGER, Constraint: PRIMARY KEY)
  - COLUMN: email (Type: VARCHAR, Tag: PII)
  - COLUMN: region_id (Type: INTEGER)

- NODE Table: ORDERS {description: "Transactional sales records", owner: "Sales"}
  - COLUMN: order_id (Type: INTEGER, Constraint: PRIMARY KEY)
  - COLUMN: customer_id (Type: INTEGER, Constraint: FOREIGN KEY)
  - COLUMN: product_sku (Type: VARCHAR, Constraint: FOREIGN KEY)
  - COLUMN: total_amount (Type: DECIMAL)

- NODE Table: PRODUCTS {description: "Inventory catalog", owner: "Warehouse"}
  - COLUMN: sku (Type: VARCHAR, Constraint: PRIMARY KEY)
  - COLUMN: product_name (Type: VARCHAR)
  - COLUMN: category (Type: VARCHAR)

### EXPLICIT RELATIONSHIPS
- RELATIONSHIP: Table[ORDERS] --REFERENCES--> Table[USERS] via customer_id
- RELATIONSHIP: Table[ORDERS] --REFERENCES--> Table[PRODUCTS] via product_sku
- RELATIONSHIP: Table[USERS] --BELONGS_TO--> Node[REGION] via region_id
- LINEAGE: Table[RAW_CRM_DATA] --FLOWS_TO--> Table[USERS]
- LINEAGE: Table[POS_SYSTEM] --FLOWS_TO--> Table[ORDERS]
"""